# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 23 2022 11:01AM,240286,10,0085797880,ISDIN Corporation,Released
1,Jun 23 2022 11:01AM,240286,10,0085798016,ISDIN Corporation,Released
2,Jun 23 2022 11:01AM,240286,10,0085798024,ISDIN Corporation,Released
3,Jun 23 2022 11:01AM,240286,10,0085798045,ISDIN Corporation,Released
4,Jun 23 2022 11:01AM,240286,10,0085798087,ISDIN Corporation,Released
5,Jun 23 2022 11:01AM,240286,10,0085798079,ISDIN Corporation,Released
6,Jun 23 2022 11:01AM,240286,10,0085798098,ISDIN Corporation,Released
7,Jun 23 2022 11:01AM,240286,10,0085798105,ISDIN Corporation,Released
8,Jun 23 2022 11:01AM,240286,10,0085798106,ISDIN Corporation,Released
9,Jun 23 2022 11:01AM,240286,10,0085798111,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,240282,Executing,6
31,240282,Released,3
32,240283,Released,1
33,240285,Released,1
34,240286,Released,11


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240281,NaN,NaN,1.0
240282,NaN,6.0,3.0
240283,NaN,NaN,1.0
240285,NaN,NaN,1.0
240286,NaN,NaN,11.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240135,0.0,0.0,1.0
240194,0.0,0.0,1.0
240201,0.0,0.0,1.0
240202,5.0,1.0,1.0
240211,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240135,0,0,1
240194,0,0,1
240201,0,0,1
240202,5,1,1
240211,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240135,0,0,1
1,240194,0,0,1
2,240201,0,0,1
3,240202,5,1,1
4,240211,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240135,,,1
1,240194,,,1
2,240201,,,1
3,240202,5,1,1
4,240211,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 23 2022 11:01AM,240286,10,ISDIN Corporation
11,Jun 23 2022 10:56AM,240285,19,"Granules Pharmaceuticals, Inc."
12,Jun 23 2022 10:49AM,240279,15,"Person & Covey, Inc."
26,Jun 23 2022 10:38AM,240277,15,"Virtus Pharmaceuticals, LLC"
29,Jun 23 2022 10:34AM,240276,15,"Carwin Pharmaceutical Associates, LLC"
33,Jun 23 2022 10:34AM,240273,15,"Chadwick Pharmaceuticals, Inc."
34,Jun 23 2022 10:21AM,240283,12,Hush Hush
35,Jun 23 2022 10:18AM,240282,12,Hush Hush
44,Jun 23 2022 10:17AM,240281,10,"Herbivore Botanicals, LLC."
45,Jun 23 2022 9:41AM,240275,10,Bio-PRF


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 23 2022 11:01AM,240286,10,ISDIN Corporation,,,11
1,Jun 23 2022 10:56AM,240285,19,"Granules Pharmaceuticals, Inc.",,,1
2,Jun 23 2022 10:49AM,240279,15,"Person & Covey, Inc.",,,14
3,Jun 23 2022 10:38AM,240277,15,"Virtus Pharmaceuticals, LLC",,,3
4,Jun 23 2022 10:34AM,240276,15,"Carwin Pharmaceutical Associates, LLC",,,4
5,Jun 23 2022 10:34AM,240273,15,"Chadwick Pharmaceuticals, Inc.",,,1
6,Jun 23 2022 10:21AM,240283,12,Hush Hush,,,1
7,Jun 23 2022 10:18AM,240282,12,Hush Hush,,6,3
8,Jun 23 2022 10:17AM,240281,10,"Herbivore Botanicals, LLC.",,,1
9,Jun 23 2022 9:41AM,240275,10,Bio-PRF,,10,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 23 2022 11:01AM,240286,10,ISDIN Corporation,11,,
1,Jun 23 2022 10:56AM,240285,19,"Granules Pharmaceuticals, Inc.",1,,
2,Jun 23 2022 10:49AM,240279,15,"Person & Covey, Inc.",14,,
3,Jun 23 2022 10:38AM,240277,15,"Virtus Pharmaceuticals, LLC",3,,
4,Jun 23 2022 10:34AM,240276,15,"Carwin Pharmaceutical Associates, LLC",4,,
5,Jun 23 2022 10:34AM,240273,15,"Chadwick Pharmaceuticals, Inc.",1,,
6,Jun 23 2022 10:21AM,240283,12,Hush Hush,1,,
7,Jun 23 2022 10:18AM,240282,12,Hush Hush,3,6,
8,Jun 23 2022 10:17AM,240281,10,"Herbivore Botanicals, LLC.",1,,
9,Jun 23 2022 9:41AM,240275,10,Bio-PRF,4,10,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 23 2022 11:01AM,240286,10,ISDIN Corporation,11,,
1,Jun 23 2022 10:56AM,240285,19,"Granules Pharmaceuticals, Inc.",1,,
2,Jun 23 2022 10:49AM,240279,15,"Person & Covey, Inc.",14,,
3,Jun 23 2022 10:38AM,240277,15,"Virtus Pharmaceuticals, LLC",3,,
4,Jun 23 2022 10:34AM,240276,15,"Carwin Pharmaceutical Associates, LLC",4,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 23 2022 11:01AM,240286,10,ISDIN Corporation,11.0,NaN,NaN
1,Jun 23 2022 10:56AM,240285,19,"Granules Pharmaceuticals, Inc.",1.0,NaN,NaN
2,Jun 23 2022 10:49AM,240279,15,"Person & Covey, Inc.",14.0,NaN,NaN
3,Jun 23 2022 10:38AM,240277,15,"Virtus Pharmaceuticals, LLC",3.0,NaN,NaN
4,Jun 23 2022 10:34AM,240276,15,"Carwin Pharmaceutical Associates, LLC",4.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 23 2022 11:01AM,240286,10,ISDIN Corporation,11.0,0.0,0.0
1,Jun 23 2022 10:56AM,240285,19,"Granules Pharmaceuticals, Inc.",1.0,0.0,0.0
2,Jun 23 2022 10:49AM,240279,15,"Person & Covey, Inc.",14.0,0.0,0.0
3,Jun 23 2022 10:38AM,240277,15,"Virtus Pharmaceuticals, LLC",3.0,0.0,0.0
4,Jun 23 2022 10:34AM,240276,15,"Carwin Pharmaceutical Associates, LLC",4.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2642742,61.0,25.0,0.0
12,480565,4.0,6.0,0.0
15,961105,22.0,0.0,0.0
16,960940,4.0,0.0,0.0
19,720792,4.0,0.0,0.0
20,1201212,21.0,1.0,5.0
21,240211,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2642742,61.0,25.0,0.0
1,12,480565,4.0,6.0,0.0
2,15,961105,22.0,0.0,0.0
3,16,960940,4.0,0.0,0.0
4,19,720792,4.0,0.0,0.0
5,20,1201212,21.0,1.0,5.0
6,21,240211,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,61.0,25.0,0.0
1,12,4.0,6.0,0.0
2,15,22.0,0.0,0.0
3,16,4.0,0.0,0.0
4,19,4.0,0.0,0.0
5,20,21.0,1.0,5.0
6,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,61.0
1,12,Released,4.0
2,15,Released,22.0
3,16,Released,4.0
4,19,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,5.0,0.0
Executing,25.0,6.0,0.0,0.0,0.0,1.0,0.0
Released,61.0,4.0,22.0,4.0,4.0,21.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,5.0,0.0
1,Executing,25.0,6.0,0.0,0.0,0.0,1.0,0.0
2,Released,61.0,4.0,22.0,4.0,4.0,21.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,5.0,0.0
1,Executing,25.0,6.0,0.0,0.0,0.0,1.0,0.0
2,Released,61.0,4.0,22.0,4.0,4.0,21.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()